In [1]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 
#!import banking/ContaCorrente.cs 
#!import banking/Boleto.cs 
#!import banking/BankFunctions.cs 
#!import functioncalling/FunctionCalling.cs 


// Import Semantic Kernel
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
using Azure.AI.OpenAI;
var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();


var endpoint = new Uri(azureEndpoint);
var credentials = new Azure.AzureKeyCredential(apiKey);
var openAIClient = new OpenAIClient(endpoint, credentials);

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.12

In [2]:
var system = 
""" 
Voce é um assistente bancário que ajuda um cliente a transacionar em sua conta. Seja solicito e educado.
""";

In [3]:
var completionOptions = new ChatCompletionsOptions
{
    MaxTokens=400,
    Temperature=1f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor = 0.95f, // Top P
    DeploymentName = model
};

In [4]:
completionOptions.Messages.Add(new ChatRequestSystemMessage(system));

completionOptions.Messages

[ Azure.AI.OpenAI.ChatRequestSystemMessage ] IsUndefined False Count 1 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.ChatRequestSystemMessage Content Voce é um assistente bancário que ajuda um cliente a transacionar em sua conta. Seja solicito e educado. Name <null> Role system

In [5]:
FunctionCalling caller = new FunctionCalling();

var contaCorrentPrincipal = new ContaCorrente("123");
contaCorrentPrincipal.Depositar(1000);

Dictionary<string, ContaCorrente> contasCorrentes = new Dictionary<string, ContaCorrente>() {
    { "André", new ContaCorrente("456") },
    { "João", new ContaCorrente("789") }
};


Boleto[] boletos = { Boleto.Criar("1", 450), Boleto.Criar("2", 150) , Boleto.Criar("3", 1500) };

BankFunctions bf = new BankFunctions(contaCorrentPrincipal, boletos, contasCorrentes);
bf.ToString()

Conta Corrente Principal - Saldo: 1000
Boletos Pendentes: 
  - Numero:1 - Valor:450 - Pago:False
  - Numero:2 - Valor:150 - Pago:False
  - Numero:3 - Valor:1500 - Pago:False
Contas Relacionadas: 
  - Nome: André - Numero:456 - Saldo:0
  - Nome: João - Numero:789 - Saldo:0


In [6]:
var funDef = caller.GetDefinitionsFromObject(bf);
foreach (var fd in funDef) {
    completionOptions.Functions.Add(fd);
}

completionOptions

Azure.AI.OpenAI.ChatCompletionsOptions ChoiceCount <null> DeploymentName gpt-4 FrequencyPenalty 0 MaxTokens 400 NucleusSamplingFactor 0.95 PresencePenalty 0 StopSequences [ ] IsUndefined True Count 0 IsReadOnly False (values) [ ] Temperature 1 TokenSelectionBiases (empty) Functions [ Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition ] IsUndefined False Count 6 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.FunctionDefinition Name Saldo Description Obtem o Saldo da Conta corrente do usuario atual Parameters { "type": "object", "properties": {}} 1 Azure.AI.OpenAI.FunctionDefinition Name ObtemBoletosPendentes Description Obtem os boletos pendentes do usuario atual Parameters { "type": "object", "properties": {}} 2 Azure.AI.OpenAI.FunctionDefinition Name PagarBoleto Description Obtem os boletos pendentes do usuario atual Parameters { "type": "object", "properties": {"numeroBoleto":{"Type":"string","Description":"Numero do Boleto"}}} 3 Azure.AI.OpenAI.FunctionDefinition Name ObterContaRelacionada Description Obtem os numeros de contas relacionadas ao usuario atual pelo nome ou apelido Parameters { "type": "object", "properties": {"nomeConta":{"Type":"string","Description":"Nome da Conta"}}} 4 Azure.AI.OpenAI.FunctionDefinition Name Transferir Description Transferir um valor para uma conta relacionada Parameters { "type": "object", "properties": {"numeroConta":{"Type":"string","Description":"Numero da Conta relacionada"}, "valor":{"Type":"string","Description":"Valor a ser transferido"}}} 5 Azure.AI.OpenAI.FunctionDefinition Name ToString Description <null> Parameters { "type": "object", "properties": {}} FunctionCall <null> AzureExtensionsOptions <null> ToolChoice <null> Messages [ Azure.AI.OpenAI.ChatRequestSystemMessage ] IsUndefined False Count 1 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.ChatRequestSystemMessage Content Voce é um assistente bancário que ajuda um cliente a transacionar em sua conta. Seja solicito e educado. Name <null> Role system User <null> Seed <null> ResponseFormat <null> Tools [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty)

In [7]:
completionOptions.Messages.Add(new ChatRequestUserMessage("Pague todos os boletos da minha conta que não estouram o saldo e transfira 100 para o João. Me diga o saldo."));


In [23]:
do {
var response = await openAIClient.GetChatCompletionsAsync(
            completionOptions);

var responseChoice = response.Value.Choices[0];
completionOptions.Messages.Add(new ChatRequestAssistantMessage(responseChoice.Message));
 if (responseChoice.FinishReason == CompletionsFinishReason.FunctionCall)
{
            // Include the FunctionCall message in the conversation history
            var funcResp = caller.ExecuteFunction(responseChoice.Message.FunctionCall, bf);
            completionOptions.Messages.Add(new ChatRequestFunctionMessage(responseChoice.Message.FunctionCall.Name, funcResp));
} else {
    Console.WriteLine(responseChoice.Message.Content);
    break;
}

}while(true);
completionOptions


ExecuteFunction ObtemBoletosPendentes called
Arguments: {}
Boletos Pendentes: 
Numero:1 - Valor:450
Numero:2 - Valor:150
Numero:3 - Valor:1500

ExecuteFunction Saldo called
Arguments: {}
Saldo: 1000
ExecuteFunction PagarBoleto called
Arguments: {
  "numeroBoleto": 2
}
Parameter numeroBoleto of type String
Element: 2
Pagar Boleto 2 chamado
ExecuteFunction Saldo called
Arguments: {}
Saldo: 850
ExecuteFunction PagarBoleto called
Arguments: {
  "numeroBoleto": 1
}
Parameter numeroBoleto of type String
Element: 1
Pagar Boleto 1 chamado
ExecuteFunction Saldo called
Arguments: {}
Saldo: 400
ExecuteFunction ObterContaRelacionada called
Arguments: {
  "nomeConta": "João"
}
Parameter nomeConta of type String
Element: João
ObterContaRelacionada João chamado
ExecuteFunction Transferir called
Arguments: {
  "numeroConta": 789,
  "valor": 100
}
Parameter numeroConta of type String
Element: 789
Parameter valor of type String
Element: 100
Transferir 100 para 789 chamado
ExecuteFunction Saldo called
Ar

Azure.AI.OpenAI.ChatCompletionsOptions ChoiceCount <null> FrequencyPenalty 0 MaxTokens 400 NucleusSamplingFactor 0.95 PresencePenalty 0 StopSequences [ ] IsUndefined True Count 0 IsReadOnly False (values) [ ] Temperature 1 TokenSelectionBiases (empty) User <null> Functions [ Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition, Azure.AI.OpenAI.FunctionDefinition ] IsUndefined False Count 6 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.FunctionDefinition Name Saldo Description Obtem o Saldo da Conta corrente do usuario atual Parameters { "type": "object", "properties": {}} 1 Azure.AI.OpenAI.FunctionDefinition Name ObtemBoletosPendentes Description Obtem os boletos pendentes do usuario atual Parameters { "type": "object", "properties": {}} 2 Azure.AI.OpenAI.FunctionDefinition Name PagarBoleto Description Obtem os boletos pendentes do usuario atual Parameters { "type": "object", "properties": {"numeroBoleto":{"Type":"string","Description":"Numero do Boleto"}}} 3 Azure.AI.OpenAI.FunctionDefinition Name ObterContaRelacionada Description Obtem os numeros de contas relacionadas ao usuario atual pelo nome ou apelido Parameters { "type": "object", "properties": {"nomeConta":{"Type":"string","Description":"Nome da Conta"}}} 4 Azure.AI.OpenAI.FunctionDefinition Name Transferir Description Transferir um valor para uma conta relacionada Parameters { "type": "object", "properties": {"numeroConta":{"Type":"string","Description":"Numero da Conta relacionada"}, "valor":{"Type":"string","Description":"Valor a ser transferido"}}} 5 Azure.AI.OpenAI.FunctionDefinition Name ToString Description <null> Parameters { "type": "object", "properties": {}} FunctionCall <null> AzureExtensionsOptions <null> Messages [ Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage ... (1 more) ] IsUndefined False Count 21 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.ChatMessage Role system Content Voce é um assistente bancário que ajuda um cliente a transacionar em sua conta. Seja solicito e educado. Name <null> FunctionCall <null> AzureExtensionsContext <null> 1 Azure.AI.OpenAI.ChatMessage Role user Content Pague todos os boletos da minha conta que não estouram o saldo e transfira 100 para o João. Me diga o saldo. Name <null> FunctionCall <null> AzureExtensionsContext <null> 2 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name ObtemBoletosPendentes Arguments {} AzureExtensionsContext <null> 3 Azure.AI.OpenAI.ChatMessage Role function Content "Boletos Pendentes: \nNumero:1 - Valor:450\nNumero:2 - Valor:150\nNumero:3 - Valor:1500\n" Name ObtemBoletosPendentes FunctionCall <null> AzureExtensionsContext <null> 4 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name Saldo Arguments {} AzureExtensionsContext <null> 5 Azure.AI.OpenAI.ChatMessage Role function Content "1000" Name Saldo FunctionCall <null> AzureExtensionsContext <null> 6 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name PagarBoleto Arguments {
 "numeroBoleto": 2
} AzureExtensionsContext <null> 7 Azure.AI.OpenAI.ChatMessage Role function Content "Boleto 2 pago com sucesso!" Name PagarBoleto FunctionCall <null> AzureExtensionsContext <null> 8 Azure.AI.OpenAI.

In [24]:
bf.ToString()

Conta Corrente Principal - Saldo: 300
Boletos Pendentes: 
  - Numero:1 - Valor:450 - Pago:True
  - Numero:2 - Valor:150 - Pago:True
  - Numero:3 - Valor:1500 - Pago:False
Contas Relacionadas: 
  - Nome: André - Numero:456 - Saldo:0
  - Nome: João - Numero:789 - Saldo:100
